# MNIST Large Untrained Net Exc Inh

Derived from https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from keras import backend as K
#print("keras version:",tf.keras.__version__)

TensorFlow version: 2.15.0


In [2]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import math

In [3]:
inhibitoryNeuronOutputPositive = True
if(inhibitoryNeuronOutputPositive):
    inhibitoryNeuronSwitchActivation = True
else:
    inlineImplementation = False	#orig: True #True: excitatory/inhibitory neurons are on same sublayer, False: add inhibitory neurons to separate preceding sublayer
    if(inlineImplementation):
        positiveWeightImplementation = False	#orig: True #optional
        if(not positiveWeightImplementation):
            integrateWeights = True    #orig: False #optional
            if(integrateWeights):
                integrateWeights1 = False    #explicitly declare E/I neurons
                integrateWeights2 = True    #implicitly declare E/I neurons
                integrateWeightsInitialiseZero = False   #miscellaneous training performance improvement (single EI layer only)
        excitatoryNeuronThreshold = 0.0 #mandatory
    else:
        positiveWeightImplementation = False    #False: only current coded implementation
        inhibitoryNeuronNormalisationFactorStatic = False    #True: normalise intermediary inhibitory neuron layer based on h0/h1 num neurons, False: normalise based on h0/h1 activations
        excitatoryNeuronThreshold = 0.0   #orig: 0.0

inputLayerExcitatoryOnly = True #True: only current coded implementation

generateUntrainedNetwork = False
if(generateUntrainedNetwork):
    #only train the last layer
    numberOfHiddenLayers = 2    #default: 2    #if 0 then useSVM=True
else:
    numberOfHiddenLayers = 2 #default: 2

if(numberOfHiddenLayers > 1):
    addSkipLayers = False   #optional
else:
    addSkipLayers = False   #mandatory

layerSizeBase = 128  #default: 128

batch_size = 64 #default: 64
epochs = 5  #100  #1  #5

debugNoEIneurons = False
debugPreTrainWeights = True
debugPreTrainOutputs = True
debugPostTrainWeights = True
debugPostTrainOutputs = True

## Load data

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("x_train.shape = ", x_train.shape)

input_shape = (28, 28)

x_train.shape =  (60000, 28, 28)


## Define model

In [5]:
num_classes = 10

In [6]:
def EIactivation(Z):
    A = K.maximum(Z, excitatoryNeuronThreshold)-excitatoryNeuronThreshold  #ReLU
    return A

def EIactivationExcitatory(Z):
    if(inlineImplementation):
        if(positiveWeightImplementation):
            return EIactivation(Z)
        else:
             print("EIactivationExcitatory error: requires positiveWeightImplementation")
    else:
        print("EIactivationExcitatory error: requires inlineImplementation")

def EIactivationInhibitory(Z):
    if(inlineImplementation):
        if(positiveWeightImplementation):
            return -EIactivation(Z)   #ReLU with negative output
        else:
             print("EIactivationInhibitory error: requires positiveWeightImplementation")
    else:
        print("inlineImplementation error: requires inlineImplementation")

def EIweightInitializer(shape, dtype=None):
    if(inlineImplementation):
        if(positiveWeightImplementation):
            w = tf.math.abs(tf.random.normal(shape, dtype=dtype))
        else:
            if(integrateWeights):
                if(integrateWeightsInitialiseZero):
                    w = tf.random.normal(shape, dtype=dtype)
                    #w = tf.zeros(shape, dtype=dtype)    #tf.math.abs(tf.random.normal(shape, dtype=dtype))
                else:
                    w = tf.math.abs(tf.random.normal(shape, dtype=dtype))
                    wEIsize = w.shape[0]//2
                    wSignE = tf.ones([wEIsize, w.shape[1]])
                    wSignI = tf.ones([wEIsize, w.shape[1]])
                    wSignI = tf.multiply(wSignI, -1)
                    wSign = tf.concat([wSignE, wSignI], axis=0)
                    w = tf.multiply(w, wSign)
            else:
                print("EIweightInitializer error: requires !positiveWeightImplementation:integrateWeights")
    else:
        print("EIweightInitializer error: requires inlineImplementation")

    return w

def EIweightInitializerExcitatory(shape, dtype=None):
    if(positiveWeightImplementation):
        print("EIweightInitializerExcitatory error: requires !positiveWeightImplementation")
    else:
        return tf.math.abs(tf.random.normal(shape, dtype=dtype))

def EIweightInitializerInhibitory(shape, dtype=None):
    if(positiveWeightImplementation):
        print("EIweightInitializerExcitatory error: requires !positiveWeightImplementation")
    else:
        return tf.math.negative(tf.math.abs(tf.random.normal(shape, dtype=dtype)))

def EIweightInitialisedAverage(shape):
    return tf.reduce_mean(tf.math.abs(tf.random.normal(shape)))

class negative(tf.keras.constraints.Constraint):
    #based on https://www.tensorflow.org/api_docs/python/tf/keras/constraints/Constraint
    def __init__(self):
        pass
    def __call__(self, w):
        return w * tf.cast(tf.math.less_equal(w, 0.), w.dtype)

class positiveOrNegative(tf.keras.constraints.Constraint):
    #based on https://www.tensorflow.org/api_docs/python/tf/keras/constraints/Constraint
    def __init__(self):
        pass
    def __call__(self, w):
        w_shape = w.shape
        wEIsize = w.shape[0]//2
        wE = w[0:wEIsize]
        wI = w[wEIsize:]
        wEcheck = tf.greater_equal(wE, 0)
        wIcheck = tf.less_equal(wI, 0)
        wEcheck = tf.cast(wEcheck, tf.float32)
        wIcheck = tf.cast(wIcheck, tf.float32)
        wE = tf.multiply(wE, wEcheck)
        wI = tf.multiply(wI, wIcheck)
        w = tf.concat([wE, wI], axis=0)
        return w


In [7]:
if(not inhibitoryNeuronOutputPositive):
    if(inlineImplementation):
        if(positiveWeightImplementation):
            EIweightConstraint = tf.keras.constraints.non_neg()
            constrainBiases = True   #ensure positive biases also
            if(constrainBiases):
                EIbiasConstraint = tf.keras.constraints.non_neg()
                constrainBiasesLastLayer = False
                if(constrainBiasesLastLayer):
                    EIbiasConstraintLastLayer = tf.keras.constraints.non_neg()
                else:
                    EIbiasConstraintLastLayer = None
            else:
                EIbiasConstraint = None
                EIbiasConstraintLastLayer = None
            EIweightConstraintLastLayer = EIweightConstraint
        else:
            if(integrateWeights):
                EIweightConstraint = positiveOrNegative()
                EIbiasConstraint = None
                EIweightConstraintLastLayer = None
                EIbiasConstraintLastLayer = None
            else:
                EIweightConstraintPositive = tf.keras.constraints.non_neg()
                EIweightConstraintNegative = negative()
                constrainBiases = False
                if(constrainBiases):
                    EIbiasConstraintPositive = tf.keras.constraints.non_neg()
                    EIbiasConstraintNegative = negative()
                else:
                    EIbiasConstraintPositive = None
                    EIbiasConstraintNegative = None
                EIweightConstraintLastLayer = None
                EIbiasConstraintLastLayer = None
    else:
        EIweightConstraintPositive = tf.keras.constraints.non_neg()
        EIweightConstraintNegative = negative()
        constrainBiases = False
        if(constrainBiases):
            EIbiasConstraintPositive = tf.keras.constraints.non_neg()
            EIbiasConstraintNegative = negative()
        else:
            EIbiasConstraintPositive = None
            EIbiasConstraintNegative = None
        EIweightConstraintLastLayer = None
        EIbiasConstraintLastLayer = None

if(generateUntrainedNetwork):
    #only train the last layer
    generateLargeNetwork = True
else:
    generateLargeNetwork = False


if(generateLargeNetwork):
    generateLargeNetworkRatio = 50
    layerRatio = generateLargeNetworkRatio
else:
    layerRatio = 1  #10 #1

def createEIlayer(layerIndex, h0, numChannels, previousNumChannels, firstLayer=False):
    if(debugNoEIneurons):
        h1 = tf.keras.layers.Dense(numChannels)(h0)
        h1 = tf.keras.layers.Activation(EIactivation)(h1)
    else:
        if(inhibitoryNeuronOutputPositive):
            h1E = tf.keras.layers.Dense(numChannels)(h0)
            h1I = tf.keras.layers.Dense(numChannels)(h0)
            h1E = tf.keras.layers.Activation('relu')(h1E)
            if(inhibitoryNeuronSwitchActivation):
                h1I = tf.keras.layers.Activation('relu')(-h1I)
            else:
                h1I = tf.keras.layers.Activation('relu')(h1I)
            h1 = tf.keras.layers.Concatenate()([h1E, h1I])
        else:
            if(inlineImplementation):
                if(positiveWeightImplementation):
                    h1E = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializerExcitatory, kernel_constraint=EIweightConstraint, bias_constraint=EIbiasConstraint)(h0)
                    h1I = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializerInhibitory, kernel_constraint=EIweightConstraint, bias_constraint=EIbiasConstraint)(h0)
                    h1E = tf.keras.layers.Activation(EIactivationExcitatory)(h1E)
                    h1I = tf.keras.layers.Activation(EIactivationInhibitory)(h1I)
                    h1 = tf.keras.layers.Concatenate()([h1E, h1I])
                else:
                    if(integrateWeights):
                        if(integrateWeights1):
                            if(firstLayer):
                                h1E = tf.keras.layers.Dense(numChannels)(h0)   #excitatory neuron inputs
                                h1I = tf.keras.layers.Dense(numChannels)(h0)   #inhibitory neuron inputs
                            else:
                                h1E = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializer, kernel_constraint=EIweightConstraint, bias_constraint=EIbiasConstraint)(h0)   #excitatory neuron inputs
                                h1I = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializer, kernel_constraint=EIweightConstraint, bias_constraint=EIbiasConstraint)(h0)   #inhibitory neuron inputs
                            h1E = tf.keras.layers.Activation(EIactivation)(h1E)
                            h1I = tf.keras.layers.Activation(EIactivation)(h1I)
                            h1 = tf.keras.layers.Concatenate()([h1E, h1I])
                        elif(integrateWeights2):
                            if(firstLayer):
                                h1 = tf.keras.layers.Dense(numChannels*2)(h0)   #excitatory neuron inputs   #OLD:*4
                            else:
                                h1 = tf.keras.layers.Dense(numChannels*2, kernel_initializer=EIweightInitializer, kernel_constraint=EIweightConstraint, bias_constraint=EIbiasConstraint)(h0)   #excitatory neuron inputs
                            h1 = tf.keras.layers.Activation(EIactivation)(h1) #ReLU
                    else:
                        if(firstLayer):
                            h1E = tf.keras.layers.Dense(numChannels)(h0)   #excitatory neuron inputs
                            h1I = tf.keras.layers.Dense(numChannels)(h0)   #inhibitory neuron inputs
                        else:
                            h0E, h0I = h0
                            h1Ee = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializerExcitatory, kernel_constraint=EIweightConstraintPositive, bias_constraint=EIbiasConstraintPositive)(h0E) #excitatory neuron excitatory inputs
                            h1Ei = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializerInhibitory, kernel_constraint=EIweightConstraintNegative, bias_constraint=EIbiasConstraintNegative)(h0I) #excitatory neuron inhibitory inputs
                            h1Ie = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializerExcitatory, kernel_constraint=EIweightConstraintPositive, bias_constraint=EIbiasConstraintPositive)(h0E) #inhibitory neuron excitatory inputs
                            h1Ii = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializerInhibitory, kernel_constraint=EIweightConstraintNegative, bias_constraint=EIbiasConstraintNegative)(h0I) #inhibitory neuron inhibitory inputs
                            h1E = tf.keras.layers.Add()([h1Ee, h1Ei])
                            h1I = tf.keras.layers.Add()([h1Ie, h1Ii])
                        h1E = tf.keras.layers.Activation(EIactivation)(h1E)
                        h1I = tf.keras.layers.Activation(EIactivation)(h1I)
                        h1 = (h1E, h1I)
            else:
                h1I = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializerExcitatory, kernel_constraint=EIweightConstraintPositive, bias_constraint=EIbiasConstraintPositive)(h0) #inhibitory interneuron (excitatory inputs)
                if(not inhibitoryNeuronNormalisationFactorStatic):  #disabled because modifies activation levels
                    h1I = tf.keras.layers.Activation(EIactivation)(h1I)
                h1I = h1I*calculateInhibitoryNeuronNormalisationFactor(h0, h1I, numChannels, previousNumChannels, firstLayer)
                h1Ee = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializerExcitatory, kernel_constraint=EIweightConstraintPositive, bias_constraint=EIbiasConstraintPositive)(h0) #excitatory neuron excitatory inputs
                h1Ei = tf.keras.layers.Dense(numChannels, kernel_initializer=EIweightInitializerInhibitory, kernel_constraint=EIweightConstraintNegative, bias_constraint=EIbiasConstraintNegative)(h1I) #excitatory neuron inhibitory inputs
                h1E = tf.keras.layers.Add()([h1Ee, h1Ei])
                h1E = tf.keras.layers.Activation(EIactivation)(h1E)
                h1 = h1E
    return h1

def calculateAverageWeight(numChannels, previousNumChannels):
    shape = [previousNumChannels,numChannels]
    averageWeight = EIweightInitialisedAverage(shape)
    #averageWeight = 1.00
    #avg = x*sqrt(pi/2) = 1.25331413732 #https://stats.stackexchange.com/questions/363240/mean-of-absgauss-as-a-function-of-the-standard-deviation
    return averageWeight

def calculateInhibitoryNeuronNormalisationFactor(h0, h1I, numChannels, previousNumChannels, firstLayer=False):
    if(inhibitoryNeuronNormalisationFactorStatic):
        averageLayerActivation = 0.5    #this is not correct
        averageWeight = calculateAverageWeight(numChannels, previousNumChannels)
        if(firstLayer):
            #assume input layer unequal activation/nonactivation level
            averageLayerActivation = np.mean(x_train)
        print("previousNumChannels = ", previousNumChannels)
        print("averageLayerActivation = ", averageLayerActivation)
        print("averageWeight = ", averageWeight)
        h1InormalisationFactor = (1/(previousNumChannels*averageWeight*averageLayerActivation))
    else:
        h1InormalisationFactor = tf.reduce_mean(h0)/tf.reduce_mean(h1I)
    return h1InormalisationFactor

def concatEIneurons(h):
    if(inhibitoryNeuronOutputPositive):
        return h
    else:
        if(inlineImplementation):
            if(positiveWeightImplementation):
                return h
            else:
                if(integrateWeights):
                    pass
                else:
                    hE, hI = h
                    h = tf.keras.layers.Concatenate()([hE, hI])
                return h
        else:
            return h

In [8]:
x = tf.keras.layers.Input(shape=input_shape)
h0 = tf.keras.layers.Flatten()(x)
hLast = h0

previousNumChannels = input_shape[0]*input_shape[1]
if(numberOfHiddenLayers >= 1):
    numChannels = layerSizeBase*layerRatio
    h1 = createEIlayer(1, h0, numChannels, previousNumChannels, firstLayer=True)
    hLast = h1
    previousNumChannels = numChannels
if(numberOfHiddenLayers >= 2):
    numChannels = layerSizeBase*layerRatio
    h2 = createEIlayer(2, h1, numChannels, previousNumChannels)
    hLast = h2
    previousNumChannels = numChannels
if(numberOfHiddenLayers >= 3):
    numChannels = layerSizeBase*layerRatio
    h3 = createEIlayer(3, h2, numChannels, previousNumChannels)
    hLast = h3
    previousNumChannels = numChannels
if(numberOfHiddenLayers >= 4):
    numChannels = layerSizeBase*layerRatio
    h4 = createEIlayer(4, h3, numChannels, previousNumChannels)
    hLast = h4
    previousNumChannels = numChannels

if(addSkipLayers):
    mList = []
    if(numberOfHiddenLayers >= 1):
        m1 = tf.keras.layers.Flatten()(concatEIneurons(h1))
        mList.append(m1)
    if(numberOfHiddenLayers >= 2):
        m2 = tf.keras.layers.Flatten()(concatEIneurons(h2))
        mList.append(m2)
    if(numberOfHiddenLayers >= 3):
        m3 = tf.keras.layers.Flatten()(concatEIneurons(h3))
        mList.append(m3)
    if(numberOfHiddenLayers >= 4):
        m4 = tf.keras.layers.Flatten()(concatEIneurons(h4))
        mList.append(m4)
    hLast = tf.keras.layers.concatenate(mList)
else:
    hLast = concatEIneurons(hLast)

if(generateUntrainedNetwork):
    hLast = tf.keras.layers.Lambda(lambda x: tf.keras.backend.stop_gradient(x))(hLast)

if(inhibitoryNeuronOutputPositive):
    y = tf.keras.layers.Dense(num_classes, activation='softmax')(hLast)
else:
    y = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_constraint=EIweightConstraintLastLayer, bias_constraint=EIbiasConstraintLastLayer)(hLast)
model = tf.keras.Model(x, y)

In [9]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [10]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    #temp: model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])

print(model.summary())
#printModelSummary(model)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28)]             0         []                            
                                                                                                  
 flatten (Flatten)           (None, 784)                  0         ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 128)                  100480    ['flatten[0][0]']             
                                                                                                  
 dense (Dense)               (None, 128)                  100480    ['flatten[0][0]']             
                                                                                              

In [11]:
if(debugPreTrainWeights):
    testwritefile = open('weightsPreTrain.txt', 'w')
    for layerIndex, layer in enumerate(model.layers):
        heading = "\n" + "layer = " + str(layerIndex) + "\n"
        testwritefile.write(heading)
        weights = layer.get_weights()
        #weightsAvg = np.mean(weights[0])
        #print(heading)
        #print("weightsAvg = ", weightsAvg)
        weightsS =  str(weights)
        testwritefile.write(weightsS)
    testwritefile.close()

In [12]:
if(debugPreTrainOutputs):
    testwritefile = open('outputPreTrain.txt', 'w')
    xTrainFirstSample = np.expand_dims(x_train[0], axis=0)
    for layerIndex, layer in enumerate(model.layers):
        heading = "\n" + "layer = " + str(layerIndex) + "\n"
        testwritefile.write(heading)
        func = K.function([model.get_layer(index=0).input], layer.output)
        layerOutput = func([xTrainFirstSample])  # input_data is a numpy array
        #print(heading)
        #print("layerOutput.shape = ", layerOutput.shape)
        layerOutputS =  str(layerOutput)
        testwritefile.write(layerOutputS)
    testwritefile.close()

## Train model

In [13]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


938/938 [==============================] - 8s 8ms/step - loss: 0.2194 - accuracy: 0.9349
Epoch 2/5
938/938 [==============================] - 7s 7ms/step - loss: 0.0871 - accuracy: 0.9727
Epoch 3/5
938/938 [==============================] - 7s 7ms/step - loss: 0.0589 - accuracy: 0.9814
Epoch 4/5
938/938 [==============================] - 7s 8ms/step - loss: 0.0432 - accuracy: 0.9864
Epoch 5/5
938/938 [==============================] - 6s 7ms/step - loss: 0.0327 - accuracy: 0.9892


In [14]:
if(debugPostTrainWeights):
    testwritefile = open('weightsPostTrain.txt', 'w')
    for layerIndex, layer in enumerate(model.layers):
        heading = "\n" + "layer = " + str(layerIndex) + "\n"
        testwritefile.write(heading)
        weights = layer.get_weights()
        #print(heading)
        #print(weights)
        weightsS =  str(weights)
        testwritefile.write(weightsS)
    testwritefile.close()

In [15]:
if(debugPostTrainOutputs):
    testwritefile = open('outputPostTrain.txt', 'w')
    xTrainFirstSample = np.expand_dims(x_train[0], axis=0)
    for layerIndex, layer in enumerate(model.layers):
        heading = "\n" + "layer = " + str(layerIndex) + "\n"
        testwritefile.write(heading)
        func = K.function([model.get_layer(index=0).input], layer.output)
        layerOutput = func([xTrainFirstSample])  # input_data is a numpy array
        #print(heading)
        #print(layerOutput)
        layerOutputS = str(layerOutput)  #tf.tensor.toString(layerOutput)    #layerOutput.tostring()
        testwritefile.write(layerOutputS)
    testwritefile.close()

## Evaluate model

In [16]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0848 - accuracy: 0.9752 - 1s/epoch - 4ms/step


[0.08482825011014938, 0.9751999974250793]

In [17]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [18]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.0853368 , 0.0853368 , 0.08533684, 0.08533705, 0.0853368 ,
        0.0853368 , 0.0853368 , 0.23196852, 0.0853368 , 0.08533682],
       [0.08533674, 0.08533677, 0.23196927, 0.08533674, 0.08533674,
        0.08533674, 0.08533674, 0.08533674, 0.08533674, 0.08533674],
       [0.08533952, 0.23192555, 0.08534102, 0.08533955, 0.08533973,
        0.0853397 , 0.08533981, 0.0853519 , 0.08534375, 0.08533952],
       [0.23152587, 0.085365  , 0.08549026, 0.08536571, 0.08536474,
        0.08542477, 0.08536538, 0.08536545, 0.08536474, 0.08536804],
       [0.08533739, 0.08533729, 0.08533733, 0.08533729, 0.23196074,
        0.08533729, 0.08533741, 0.08533802, 0.0853373 , 0.08533991]],
      dtype=float32)>